In [17]:
print(9)

9


In [18]:
from io import StringIO
import requests
import json
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
df = pd.read_csv('BTC-USD.csv',parse_dates= ["Date"],index_col = 0).dropna()
df_plot = df[{'Open','High','Low','Close'}]

In [19]:
df_plot.head()

,High,Low,Close,Open
Date,,,,
2020-03-13,5838.114746,4106.980957,5563.707031,5017.831055
2020-03-14,5625.226563,5125.069336,5200.366211,5573.077637
2020-03-15,5836.645020,5169.283203,5392.314941,5201.066895
2020-03-16,5385.229492,4575.357910,5014.479980,5385.229492
2020-03-17,5371.348633,4981.909180,5225.629395,5002.578125


In [20]:
#計算ma
def moving_average(data,periods):
    return data['Close'].rolling(periods).mean()

In [21]:
df_copy = df.copy()
df_copy['min'] = df_copy['Low'].rolling(9).min()
df_copy['max'] = df_copy['High'].rolling(9).min()
df_copy['RSV'] = (df_copy['Close'] - df_copy['min']) / (df_copy['max'] - df_copy['min'])
df_copy = df_copy.dropna()
K_list = [50]
for index,rsv in enumerate(list(df_copy['RSV'])):
    K_yesterday = K_list[index]
    K_today = 2/3 * K_yesterday + 1/3 *rsv
    K_list.append(K_today)
df_copy['K'] = K_list[1:]
D_list = [50]
for index,K in enumerate(list(df_copy['K'])):
    D_yesterday = D_list[index]
    D_today = 2/3 * D_yesterday + 1/3 *K
    D_list.append(D_today)
df_copy['D'] = D_list[1:]

In [22]:
df_copy.head()

,Open,High,Low,Close,Adj Close,Volume,min,max,RSV,K,D
Date,,,,,,,,,,,
2020-03-21,6206.521484,6378.135254,5932.823242,6185.066406,6185.066406,4.249439e+10,4106.980957,5331.833984,1.696600,33.898867,44.632956
2020-03-22,6185.558105,6359.697266,5823.713867,5830.254883,5830.254883,4.009966e+10,4575.357910,5331.833984,1.658872,23.152202,37.472704
2020-03-23,5831.374512,6443.934570,5785.004395,6416.314941,6416.314941,4.649192e+10,4575.357910,5331.833984,2.433596,16.246000,30.397136
2020-03-24,6436.642578,6789.022949,6411.066406,6734.803711,6734.803711,4.822191e+10,4575.357910,5331.833984,2.854612,11.782204,24.192159
2020-03-25,6738.716797,6892.511230,6536.926270,6681.062988,6681.062988,4.459011e+10,4981.909180,5331.833984,4.855768,9.473392,19.285903


In [23]:
#計算KD值
'''
step1 計算RSV:(今天的收盤價 - 最近9天的最低價)/(最近9天的最高價 - 最近9天的最低價)
step2 計算K值: K = 2/3 * (昨天的K值) + 1/3 * (今日RSV)
step3 計算D值: K = 2/3 * (昨天的D值) + 1/3 * (今日的K值)
'''
def KD(data):
    df_copy = data.copy()
    df_copy['min'] = df_copy['Low'].rolling(9).min()
    df_copy['max'] = df_copy['High'].rolling(9).min()
    df_copy['RSV'] = (df_copy['Close'] - df_copy['min']) / (df_copy['max'] - df_copy['min'])
    df_copy = df_copy.dropna()
    K_list = [50]
    for index,rsv in enumerate(list(df_copy['RSV'])):
        K_yesterday = K_list[index]
        K_today = 2/3 * K_yesterday + 1/3 *rsv
        K_list.append(K_today)
    df_copy['K'] = K_list[1:]
    D_list = [50]
    for index,K in enumerate(list(df_copy['K'])):
        D_yesterday = D_list[index]
        D_today = 2/3 * D_yesterday + 1/3 *K
        D_list.append(D_today)
    df_copy['D'] = D_list[1:]
    use_df = pd.merge(data,df_copy[['K','D']],left_index=True,right_index=True,how='left')
    return use_df

In [24]:
KD(df).tail(10)

,Open,High,Low,Close,Adj Close,Volume,K,D
Date,,,,,,,,
2021-03-04,50522.304688,51735.089844,47656.929688,48561.167969,48561.167969,5.234382e+10,1.547484,1.473137
2021-03-05,48527.031250,49396.429688,46542.515625,48927.304688,48927.304688,4.862593e+10,1.577075,1.507783
2021-03-06,48899.230469,49147.218750,47257.527344,48912.382813,48912.382813,3.436356e+10,1.595371,1.536979
2021-03-07,48918.679688,51384.367188,48918.679688,51206.691406,51206.691406,4.313746e+10,1.827658,1.633872
2021-03-08,51174.117188,52314.070313,49506.054688,52246.523438,52246.523438,4.859743e+10,2.082265,1.783336
2021-03-09,52272.968750,54824.117188,51981.832031,54824.117188,54824.117188,5.091223e+10,2.190816,1.919163
2021-03-10,54824.011719,57258.253906,53290.890625,56008.550781,56008.550781,5.729558e+10,2.671947,2.170091
2021-03-11,55963.179688,58091.062500,54484.593750,57805.121094,57805.121094,5.677234e+10,3.222615,2.520932
2021-03-12,57821.218750,57996.621094,55376.648438,57332.089844,57332.089844,5.568994e+10,3.529191,2.857018
